In [1]:
from tool import *
import pyautogui
import sys

# general options
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

## PO DB 업데이트

In [2]:
filepath = 'D:/Downloads/' + [file for file in os.listdir('D:/Downloads/') if 'poshThru' in file][-1] # 파일이름에 poshThru가 들어가는 파일 중에 가장 최신 파일을 선택함
print(filepath)
df = pd.read_excel(filepath, sheet_name='DataSet').iloc[:, :48]
df = df[['Model', 'PO No.', 'Ship To', 'Shipping', 'Cancel', 'PO', 'OQC Report', 'OQC Date', 'OQC Result', 'Ship', 'Issued Date', 'RSD','Ship Date', 'BL No', 'BL Status', 'Method', 'Price Term', 'Unit Price', 'Currency', 'Payment Term', 'PO1 No', 'SO No.', 'Final Destination']]
df.loc[df['PO1 No'].notnull(), 'PO1 No'] = df.loc[df['PO1 No'].notnull(),'PO1 No'].astype('int').astype('str')
df.loc[df['SO No.'].notnull(), 'SO No.'] = df.loc[df['SO No.'].notnull(), 'SO No.'].astype('int').astype('str')
df['Ship'].fillna(0, inplace=True)
df['Ship'] = df['Ship'].astype('int')
df['Issued Date'] = pd.to_datetime(df['Issued Date'])
df['RSD'] = pd.to_datetime(df['RSD']) # PO의 RSD를 Datetime 형식으로 변경
df.rename(columns={'Model':'Mapping Model.Suffix', 'Ship To':'To Site'}, inplace=True)
df1 = df.groupby(['PO No.', 'Cancel', 'Issued Date', 'Mapping Model.Suffix', 'To Site', 'RSD', 'Method', 'Price Term', 'Unit Price', 'Currency', 'Payment Term', 'PO1 No', 'SO No.', 'Final Destination'], dropna=False)[['PO']].sum().reset_index().set_index('PO No.')

D:/Downloads/poshThru_20230905091210.xls


In [3]:
with open('D:/Data/TCL PO.db', 'rb') as f:
    po_db = pickle.load(f)

In [4]:
po_db1 = po_db.copy()

In [5]:
prev_po = po_db.shape[0]
prev_po

21

In [6]:
for i in df1.index:
    po_db.loc[i, :] = df1.loc[i, :]

In [7]:
updated_po = po_db.shape[0]
updated_po

21

In [8]:
add_po = po_db.drop(index=po_db1.index)

In [9]:
print(f"TCL의 PO DB를 업데이트 하였습니다. PO는 {updated_po - prev_po} 건 추가 되었습니다. ")
add_po['RSD Week'] = add_po['RSD'].map(get_weekname)
add_po.pivot_table('PO', index=['PO No.', 'To Site', 'Mapping Model.Suffix'], columns='RSD Week', aggfunc=sum).reset_index()

TCL의 PO DB를 업데이트 하였습니다. PO는 0 건 추가 되었습니다. 


RSD Week,PO No.,To Site,Mapping Model.Suffix


In [10]:
with open('D:/Data/TCL PO.db', 'wb') as f:
    pickle.dump(po_db, f)
with open('D:/Data/DB backup/TCL PO.db', 'wb') as f: # PO DB 백업용
    pickle.dump(po_db, f)

## Shipment result DB 업데이트

In [12]:
with open('D:/Data/TCL shipment result.db', 'rb') as f:
    SR_df = pickle.load(f)
SR_df1 = SR_df.copy()
df = df[df['Ship Date'].notnull()]
df['Ship Date'] = pd.to_datetime(df['Ship Date'])
df = df.set_index(['PO No.', 'BL No'])

prev_sr = SR_df.shape[0]
for i in df.index:
    SR_df.loc[i, :] = df.loc[i, :]
updated_sr = SR_df.shape[0]

In [13]:
PO = po_db[po_db['Cancel']=='N']
df = pd.concat([po_db[po_db['Cancel'] == 'N']['PO'], SR_df.groupby('PO No.').sum('Ship')['Ship']], axis=1)
df.fillna(0, inplace=True)
df['Open'] = df['PO'] - df['Ship']

In [14]:
if df[df['Open'] < 0].shape[0] > 0:
    print(df)
    sys.exit()
else:
    with open('D:/Data/TCL shipment result.db', 'wb') as f:
        pickle.dump(SR_df, f)
    with open('D:/Data/DB backup/TCL shipment result.db', 'wb') as f:
        pickle.dump(SR_df, f)
    print(f"TCL의 SR DB를 업데이트 하였습니다. 선적은 {updated_sr - prev_sr} 건 추가 되었습니다. ")

TCL의 SR DB를 업데이트 하였습니다. 선적은 0 건 추가 되었습니다. 


In [15]:
with open('D:/Data/sitemap.db', 'rb') as f:
    sitemap = pickle.load(f)

In [16]:
add_sr = SR_df.drop(index=SR_df1.index)
if add_sr.shape[0] == 0:
    print('No shipment added')
else:
    add_sr = pd.merge(add_sr.reset_index(), sitemap[['To Site', 'Region', 'Country']], how='left')
    add_sr['Ship Week'] = add_sr['Ship Date'].map(get_weekname)
    print(add_sr.pivot_table('Ship', index=['Region', 'Country', 'BL No', 'Mapping Model.Suffix', 'PO No.'], columns=['Ship Week', 'Ship Date'], aggfunc=sum).fillna(0)) # 추가된 선적 실적

No shipment added
